<a href="https://colab.research.google.com/github/govind17/Information-Retrieval-Project/blob/main/LDA_feature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U Flask
!pip install git+https://github.com/boudinfl/pke.git
!pip install matplotlib
!python -m spacy download en_core_web_sm

In [1]:
# One time installation
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/GovindShukla/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/GovindShukla/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
import numpy as np
import pandas as pd
import operator
import io
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import spacy

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize

import flask
from flask import request
from flask_cors import CORS
from flask import Flask, Response
from keyPhrasification import key_phrasification

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from matplotlib.figure import Figure

from io import BytesIO

import os
import joblib
# import pyterrier as pt
# from pyterrier_t5 import MonoT5ReRanker
import pandas as pd
import re
from nltk.tokenize import sent_tokenize, word_tokenize
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
import ast
from keyPhrasification import key_phrasification

In [ ]:
# Indexing the cord-19 dataset
if not pt.started():
    pt.init()

cord19 = pt.datasets.get_dataset('irds:cord19/trec-covid')
pt_index_path = 'C:/Users/Pritha/Desktop/SUBJECTS/PROJECT/Relevance feedback with XAI/Backend Code/Information-Retrieval-Project/terrier_cord19'
if not os.path.exists(pt_index_path + "/data.properties"):
    # create the index, using the IterDictIndexer indexer
    indexer = pt.index.IterDictIndexer(pt_index_path)
    # we give the dataset get_corpus_iter() directly to the indexer
    # while specifying the fields to index and the metadata to record
    index_ref = indexer.index(cord19.get_corpus_iter(),
                              fields=('abstract',),
                              meta=('docno',))
else:
    # if you already have the index, use it.
    index_ref = pt.IndexRef.of(pt_index_path + "/data.properties")
    index = pt.IndexFactory.of(index_ref)


In [ ]:
# Rocchio algorithm
def rocchio_algorithm(query_doc_vector, docs_relevant_vectors,
                      docs_irrelevant_vectors, key_relevant_vectors,
                      key_irrelevant_vectors,
                      alpha, beta, gamma, delta):

    # sum_of_rel_doc_vectors = docs_relevant_vectors.sum(axis=0)
    # print('sum_of_rel_doc_vectors', sum_of_rel_doc_vectors)
    sum_of_rel_doc_vectors=[]
    sum_of_irrel_doc_vectors=[]
    sum_of_rel_key_vectors=[]
    sum_of_irrel_key_vectors=[]
    for each_rel_doc_vector in docs_relevant_vectors:
        if len(sum_of_rel_doc_vectors) == 0:
            print('Length of sum_of_rel_doc_vectors is zero')
            sum_of_rel_doc_vectors = each_rel_doc_vector
        else:
            # print(each_rel_doc_vector)
            sum_of_rel_doc_vectors = list(map(operator.add, sum_of_rel_doc_vectors, each_rel_doc_vector))
            # print('SUM :', sum_of_rel_doc_vectors)
    for each_irrel_doc_vector in docs_irrelevant_vectors:
        if len(sum_of_irrel_doc_vectors) == 0:
            print('Length of sum_of_irrel_doc_vectors is zero')
            sum_of_irrel_doc_vectors = each_irrel_doc_vector
        else:
            sum_of_irrel_doc_vectors = list(map(operator.add, sum_of_irrel_doc_vectors, each_irrel_doc_vector))
    # sum_of_irrel_doc_vectors = docs_irrelevant_vectors.sum(axis=0)
    # print('sum_of_irrel_doc_vectors', sum_of_irrel_doc_vectors)
    for each_rel_key_vector in key_relevant_vectors:
        if len(sum_of_rel_key_vectors) == 0:
            print('Length of sum_of_rel_key_vectors is zero')
            sum_of_rel_key_vectors = each_rel_key_vector
        else:
            sum_of_rel_key_vectors = list(map(operator.add, sum_of_rel_key_vectors, each_rel_key_vector))
    # sum_of_rel_key_vectors = key_relevant_vectors.sum(axis=0)
    # print('sum_of_rel_key_vectors', sum_of_rel_key_vectors)
    for each_irrel_key_vector in key_irrelevant_vectors:
        if len(sum_of_irrel_key_vectors) == 0:
            print('Length of sum_of_irrel_key_vectors is zero')
            sum_of_irrel_key_vectors = each_irrel_key_vector
        else:
            sum_of_irrel_key_vectors = list(map(operator.add, sum_of_irrel_key_vectors, each_irrel_key_vector))
    # sum_of_irrel_key_vectors = key_irrelevant_vectors.sum(axis=0)
    # print('sum_of_irrel_key_vectors', (delta/len(docs_relevant_vectors)) * np.array(sum_of_irrel_key_vectors))

    new_doc_vector_query = np.sum([np.array(query_doc_vector)
    , (alpha/len(docs_relevant_vectors)) * np.array(sum_of_rel_doc_vectors)
    , (beta/len(docs_irrelevant_vectors)) * np.array(sum_of_irrel_doc_vectors)
    , (gamma/len(docs_relevant_vectors)) * np.array(sum_of_rel_key_vectors)
    , (delta/len(docs_relevant_vectors)) * np.array(sum_of_irrel_key_vectors)], axis=0)
    df = pd.DataFrame({"a": [new_doc_vector_query]})
    return df.values

# Compute cosine scores
def compute_cosine_sim(new_query_vector, all_data, name):
    consine_similarities = []
    for index, data in all_data.iterrows():
        cosine_sim = cosine_similarity([np.array(new_query_vector[0])], [np.array(all_data[name][index])])
        consine_similarities.append(cosine_sim[0][0])
    return consine_similarities

# Rank data
def rank_data(new_query_vector, dataset, name):
  cosine_sim_values = compute_cosine_sim(new_query_vector, dataset, name)
  dataset['Cosine_Similarity_' + name] = cosine_sim_values
  sorted_dataset = dataset.sort_values(by=['Cosine_Similarity_' + name], ascending=False)

  return sorted_dataset



In [ ]:
#text preprocessing
def pre_processing_data(data):
    ps = PorterStemmer()
    tagged_dataset = [TaggedDocument(words=[ps.stem(w) for w in nltk.word_tokenize(str(_d)) if word_tokenize(str(_d).lower()) not in stopwords.words('english')], tags=[str(i)]) for i, _d in enumerate(data)]
    #tagged_dataset = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]
    return tagged_dataset

def model_doc2vec(model, tagged_data, num_epochs):
    model.build_vocab(tagged_data)
    model.train(tagged_data,total_examples=len(tagged_data), epochs=num_epochs)
    return model

def vector_for_learning(model, input_docs):
    sents = input_docs
    targets, feature_vectors = zip(*[(doc.tags[0], model.infer_vector(doc.words)) for doc in sents])
    return targets, feature_vectors

def define_d2v_model(dataset):
    preprocessed_tagged_dataset = pre_processing_data(dataset)
    model_d2v = Doc2Vec(dm=1, vector_size=100, window = 10, negative=5, hs=0, min_count=2, sample = 0, alpha=0.025, min_alpha=0.001, dm_mean = 0, dbow_words=1)
    model = model_doc2vec(model_d2v, preprocessed_tagged_dataset, 100)
    return model

def get_vectors(input_data, model):
    preprocessed_tagged_input_data= pre_processing_data(input_data)
    id, vectors = np.array(vector_for_learning(model, preprocessed_tagged_input_data), dtype=object)
    return vectors

In [ ]:
monoT5 = MonoT5ReRanker(text_field='abstract')
dataset = pt.datasets.get_dataset('irds:cord19/trec-covid')
cord19_df = pd.read_csv('C:/Users/Pritha/Desktop/SUBJECTS/PROJECT/Relevance feedback with XAI/Backend Code/Information-Retrieval-Project/cord19_sum_key.csv')
# model = define_d2v_model(cord19_df['abstract'])
model = joblib.load('gensim_model_2.pkl')
query_df = pd.DataFrame
reranked_df = pd.DataFrame

def search_query(query):
  index_ref2 = pt.IndexRef.of(pt_index_path + "/data.properties")
  index2 = pt.IndexFactory.of(index_ref2)
  # print(query)
  if not pt.started():
      pt.init()
  br = pt.BatchRetrieve(index2) % 15
  pipeline = (br >> pt.text.get_text(dataset, 'abstract') >> monoT5)
  search_result = pipeline.search(query)
  search_result = search_result.drop_duplicates(subset=['docno'])
  print(search_result)
  filtered_docs = pd.merge(cord19_df, search_result, on = "docno", how = "inner")

  return filtered_docs.head(10)


# search_query('covid')

In [ ]:
def get_reranking(feedback_df : pd.DataFrame):
    global  query_df
    global reranked_df
    feedback_df["summary_vec"] = get_vectors(feedback_df['summary'], model)
    feedback_df["keyList_vec"] = get_vectors(feedback_df['KeyList'], model)
    relevant_df = feedback_df.loc[feedback_df['relevant'] == True]
    irrelevant_df = feedback_df.loc[feedback_df['relevant'] == False]
    query_df["query_vec"]  = get_vectors(query_df['query'].values, model)
    query_df['new_query_vec'] = rocchio_algorithm(query_df["query_vec"][0], relevant_df["summary_vec"].values, irrelevant_df["summary_vec"].values, relevant_df["keyList_vec"].values, irrelevant_df["keyList_vec"].values, 1.0, 0.5,1.0, 0.5)
    sorted_dataset = rank_data(query_df['new_query_vec'].values, feedback_df, "keyList_vec")
    return sorted_dataset

def generate_plot():
    rel_docs = []
    irrel_docs = []
    # plt.canvas.flush_events()
    for i, data in enumerate(reranked_df.loc[reranked_df['relevant'] == True]['summary_vec'].values):
        rel_docs.append(data)
    for i, data in enumerate(reranked_df.loc[reranked_df['relevant'] == False]['summary_vec'].values):
        irrel_docs.append(data)
    pca = PCA(n_components=2)
    pca_rel = pca.fit_transform(rel_docs)
    pca_irrel = pca.fit_transform(irrel_docs)
    query =np.concatenate(([np.array(query_df.iloc[0]['query_vec'])], [np.array(query_df.iloc[0]['new_query_vec'])]), axis=0)
    pca_query = pca.fit_transform(query)

    # plt.scatter(pca_rel[:,0], pca_rel[:,1], marker="8", label="rel_doc")
    # plt.scatter(pca_irrel[:,0], pca_irrel[:,1], marker="*", label="irrel_doc")
    # plt.scatter(pca_query[0][0], pca_query[0][1], marker="s", label="old_query")
    # plt.scatter(pca_query[1][0], pca_query[1][1], marker="s", label="new_query")
    # plt.legend()
    # fileName = 'plot.png'
    # if os.path.exists(fileName):
    #     os.remove(fileName)
    #
    # plt.savefig(fileName, format='png')
    # return plt
    fig = Figure()
    axis = fig.add_subplot(1, 1, 1)
    axis.scatter(pca_rel[:,0], pca_rel[:,1], marker="8", label="rel_doc")
    axis.scatter(pca_irrel[:,0], pca_irrel[:,1], marker="*", label="irrel_doc")
    axis.scatter(pca_query[0][0], pca_query[0][1], marker="s", label="old_query")
    axis.scatter(pca_query[1][0], pca_query[1][1], marker="s", label="new_query")
    axis.legend()

    return fig

In [ ]:
# function to extract 5 words before and after a given word
def extract_context(text, keyword):
    # split the text into sentences using NLTK's sent_tokenize() function
    sentences = sent_tokenize(text)
    # iterate through each sentence
    context = []
    for sentence in sentences:
        # split the sentence into words using NLTK's word_tokenize() function
        words = word_tokenize(sentence)
        # check if the keyword is present in the sentence
        if keyword in words:
            # find the index of the keyword in the sentence
            keyword_index = words.index(keyword)
            # find the start and end indices for the context
            start_index = max(0, keyword_index - 5)
            end_index = min(len(words), keyword_index + 6)
            # extract the context and join the words together
            context.append(' '.join(words[start_index:end_index]))
    # join the contexts for each sentence together
    return ' '.join(context) if context else ''

def extract_context_from_keylist(keylist, abstract):
    result = []
    for key in keylist:
        result.append(extract_context(abstract, key))
    return result

In [ ]:
# cord19_df = pd.read_csv('/Users/GovindShukla/Downloads/cord19_sum_key.csv')
#
# model = define_d2v_model(cord19_df[~cord19_df['abstract'].isna()]['abstract'])

In [ ]:
#
# # Save the model as a pickle in a file
# joblib.dump(model, 'gensim_model_2.pkl')
# model = joblib.load('/Users/GovindShukla/Desktop/Information-Retrieval-Project/gensim_model_2.pkl')

In [ ]:
import base64
app = flask.Flask(__name__)

# app.config["DEBUG"] = True
CORS(app)
from flask import send_file
@app.route('/query', methods=['GET'])
def search():
    query = request.args.get('searchString')
    print(query)
    searchResults = search_query(query)
    # print(searchResults.head())
    # searchResultswithkeys = key_phrasification(searchResults)
    print(searchResults)
    global query_df
    query_dict = {'query': [query], 'query_vec' : [np.nan], 'new_query_vec' : [np.nan] }
    query_df = pd.DataFrame(data=query_dict, index=[0])
    # print(searchResultswithkeys)
    # searchResultswithkeys.to_csv('search_result.csv')
    return searchResults.to_json(orient='records')

# Feedback
@app.route('/feedback', methods=['POST'])
def fetchFeedback():
    global reranked_df
    feedbackJson = request.json['updates']
    relevanceList = []
    if len(feedbackJson):
        for doc in feedbackJson:
            for relevance in doc['value']:
                relevanceList.append(relevance)
    feedback_df = pd.DataFrame(relevanceList)
    feedback_df.drop(columns=['bntStyle'], inplace=True)
    reranked_df = get_reranking(feedback_df)
    return reranked_df.to_json(orient='records')

@app.route('/plot')
def plot():
    # Generate the plot


    fig = generate_plot()
    canvas = FigureCanvas(fig)
    output = io.BytesIO()
    canvas.print_png(output)
    response = Response(output.getvalue(), mimetype='image/png')

    return response
    # buffer.seek(0)

    # image_png = buffer.getvalue()
    # buffer.close()
    # graphic = base64.b64encode(image_png).decode('utf-8')
    # graphic = graphic.decode('utf-8')
    # fig.savefig('plot.png')

    # return send_file('plot.png', mimetype='image/png')

if __name__ == "__main__":

    app.run(host='0.0.0.0',port=5117)

###Feedback

In [ ]:
def get_reranking(feedback_df : pd.DataFrame, model):
    global  query_df
    feedback_df["summary_vec"] = get_vectors(feedback_df['summary'], model)
    feedback_df["keyList_vec"] = get_vectors(feedback_df['KeyList'], model)
    relevant_df = feedback_df.loc[feedback_df['relevant'] == True]
    irrelevant_df = feedback_df.loc[feedback_df['relevant'] == False]
    query_df["query_vec"]  = get_vectors(query_df['query'].values, model)
    query_df['new_query_vec'] = rocchio_algorithm(query_df["query_vec"][0], relevant_df["summary_vec"].values, irrelevant_df["summary_vec"].values, relevant_df["keyList_vec"].values, irrelevant_df["keyList_vec"].values, 1.0, 0.5,1.0, 0.5)
    sorted_dataset = rank_data(query_df['new_query_vec'].values, feedback_df, "keyList_vec")
    return sorted_dataset

In [ ]:
def cal_RM():
    data = pd.read_csv('/Users/GovindShukla/Desktop/Information-Retrieval-Project/search_result.csv')
    print(data['summary'])
    model = define_d2v_model(data['abstract'])
    q = {'query': ['Covid19'], 'query_vec' : [np.nan], 'new_query_vec' : [np.nan] }
    query_df = pd.DataFrame(data=q, index=[0])
    data["summary_vec"] = get_vectors(data['summary'], model)
    data["keyList_vec"] = get_vectors(data['KeyList'], model)
    query_df["query_vec"]  = get_vectors(query_df['query'].values, model)
    query_df['new_query_vec'] = rocchio_algorithm(query_df["query_vec"][0], data["summary_vec"].head(3).values, data["summary_vec"].head(7).values,
                      data["keyList_vec"].head(3).values, data["keyList_vec"].head(7).values, 1.0, 0.5,1.0, 0.5)
    sorted_dataset = rank_data(query_df['new_query_vec'].values, data, "keyList_vec")
    print(sorted_dataset['summary'])
cal_RM()

In [ ]:
rel_docs = []
irrel_docs = []
for i, data in enumerate(reranked_df.loc[reranked_df['relevant'] == True]['summary_vec'].values):
    rel_docs.append(data)
for i, data in enumerate(reranked_df.loc[reranked_df['relevant'] == False]['summary_vec'].values):
    irrel_docs.append(data)


In [ ]:
reranked_df.loc[reranked_df['relevant'] == False]

In [ ]:
reranked_df.head(3)

In [ ]:
def generate_plot():
    rel_docs = []
    irrel_docs = []
    for i, data in enumerate(reranked_df.loc[reranked_df['relevant'] == True]['summary_vec'].values):
        rel_docs.append(data)
    for i, data in enumerate(reranked_df.loc[reranked_df['relevant'] == False]['summary_vec'].values):
        irrel_docs.append(data)
    pca = PCA(n_components=2)
    pca_rel = pca.fit_transform(rel_docs)
    pca_irrel = pca.fit_transform(irrel_docs)
    print(query_df)
    query =np.concatenate(([np.array(query_df.iloc[0]['query_vec'])], [np.array(query_df.iloc[0]['new_query_vec'])]), axis=0)
    pca_query = pca.fit_transform(query)

    plt.scatter(pca_rel[:,0], pca_rel[:,1], marker="8", label="rel_doc")
    plt.scatter(pca_irrel[:,0], pca_irrel[:,1], marker="*", label="irrel_doc")
    plt.scatter(pca_query[0][0], pca_query[0][1], marker="s", label="old_query")
    plt.scatter(pca_query[1][0], pca_query[1][1], marker="s", label="new_query")
    plt.legend()

    return plt

generate_plot()
